A simplified workthrough of Mask RCNN training. Currently contains no evaluation.

In [15]:
import os
import sys
import itertools
from statistics import mean
from time import time
from tqdm import tqdm
sys.path.append('..')

Set XLA to fusible. This will fuse smaller operations, but leave larger operations as is. So far, this seems to give the biggest performance gains.

In [13]:
os.environ['TF_XLA_FLAGS'] = "--tf_xla_auto_jit=fusible"

Disable all TF 2 behavior. Set TF to use GPU 0.

In [14]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.disable_v2_behavior()
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

import horovod.tensorflow as hvd
hvd.init()

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[hvd.rank()], 'GPU')
devices = tf.config.list_logical_devices('GPU')

In [4]:
hvd.rank()

0

In [16]:
from mask_rcnn.hyperparameters import dataset_params
from mask_rcnn.hyperparameters import mask_rcnn_params
from mask_rcnn import dataset_utils
import load_weights, model

Train data can be found at s3://[redacted]/data/coco/tfr_anchor/

In [17]:
train_file_pattern = '/home/ubuntu/data/train*'
batch_size = 6
data_params = dataset_params.get_data_params()
params = mask_rcnn_params.default_config().values()
data_params['batch_size'] = batch_size
params['finetune_bn'] = False
params['train_batch_size'] = batch_size
params['l2_weight_decay'] = 1e-4
params['init_learning_rate'] = 1e-4 * batch_size
params['warmup_learning_rate'] = 1e-3 * batch_size
params['warmup_steps'] = 500
params['learning_rate_steps'] = [30000,40000]
params['learning_rate_levels'] = [1e-4 * batch_size, 1e-5 * batch_size]
params['momentum'] = 0.9
params['use_batched_nms'] = True

Fast data loader is a modified data loader that handles precomputed box matches.

In [18]:
train_input_fn = dataset_utils.FastDataLoader(train_file_pattern, data_params)
train_tdf = train_input_fn(data_params)
tdf_iter = train_tdf.make_initializable_iterator()
features, labels = tdf_iter.get_next()

[MaskRCNN] INFO    : Using Dataset Sharding with Horovod


In [19]:
train_op, total_loss = model.model(features, params, labels)
model_output, predictions = model.model(features, params, labels, False)
print(train_op)

name: "DistributedMomentumOptimizer"
op: "NoOp"
input: "^cond_1/Merge"
input: "^cond/Merge"



In [25]:
model_output, predictions = model.model(features, params, labels, False)

test = predictions['detection_boxes']
#initialize the variable
#test = tf.Variable( tf.truncated_normal([2,3],stddev = 0.1))
init_op = tf.global_variables_initializer()

#tf.print(test, output_stream=sys.stderr)
#run the graph
with tf.Session() as sess:
    sess.run(tdf_iter.initializer)
    sess.run(init_op) #execute init_op
   #print the random values that we sample
    print (sess.run(test))

[[[1.0994154  1.0994154  1.0994154  1.0994154 ]
  [1.0994154  1.0994154  1.0994154  1.0994154 ]
  [1.0994154  1.0994154  1.0994154  1.0994154 ]
  ...
  [1.0994154  1.0994154  1.0994154  1.0994154 ]
  [1.0994154  1.0994154  1.0994154  1.0994154 ]
  [1.0994154  1.0994154  1.0994154  1.0994154 ]]

 [[1.4615387  1.4615387  1.4615387  1.4615387 ]
  [1.4615387  1.4615387  1.4615387  1.4615387 ]
  [1.4615387  1.4615387  1.4615387  1.4615387 ]
  ...
  [1.4615387  1.4615387  1.4615387  1.4615387 ]
  [1.4615387  1.4615387  1.4615387  1.4615387 ]
  [1.4615387  1.4615387  1.4615387  1.4615387 ]]

 [[0.73333335 0.73333335 0.73333335 0.73333335]
  [0.73333335 0.73333335 0.73333335 0.73333335]
  [0.73333335 0.73333335 0.73333335 0.73333335]
  ...
  [0.73333335 0.73333335 0.73333335 0.73333335]
  [0.73333335 0.73333335 0.73333335 0.73333335]
  [0.73333335 0.73333335 0.73333335 0.73333335]]

 [[0.94392526 0.94392526 0.94392526 0.94392526]
  [0.94392526 0.94392526 0.94392526 0.94392526]
  [0.94392526 0.

In [10]:
predictions

{'num_detections': <tf.Tensor 'generate_detections_1/combined_non_max_suppression/CombinedNonMaxSuppression:3' shape=(6,) dtype=int32>,
 'detection_boxes': <tf.Tensor 'generate_detections_1/denormalize_box/concat:0' shape=(6, 100, 4) dtype=float32>,
 'detection_classes': <tf.Tensor 'generate_detections_1/add:0' shape=(6, 100) dtype=float32>,
 'detection_scores': <tf.Tensor 'generate_detections_1/combined_non_max_suppression/CombinedNonMaxSuppression:1' shape=(6, 100) dtype=float32>,
 'detection_masks': <tf.Tensor 'Sigmoid_1:0' shape=(6, 100, 28, 28) dtype=float32>,
 'source_id': <tf.Tensor 'IteratorGetNext:2' shape=(6,) dtype=int64>,
 'image_info': <tf.Tensor 'IteratorGetNext:0' shape=(6, 5) dtype=float32>}

Load pretrained resnet weights. Make sure to first run the ./download_and_process_pretrained_weights.sh script.

In [20]:
var_list = load_weights.build_assigment_map('resnet50/')
checkpoint_file = tf.train.latest_checkpoint('../resnet/resnet-nhwc-2018-02-07/')
_init_op, _init_feed_dict = load_weights.assign_from_checkpoint(checkpoint_file, var_list)

Training loop. To run multi-gpu with horovod, use the run.sh script.

In [27]:
var_initializer = tf.global_variables_initializer()
loss_history = []
with tf.Session() as sess:
    sess.run(_init_op, _init_feed_dict)
    sess.run(tdf_iter.initializer)
    sess.run(var_initializer)
    #progressbar = tqdm(range(10000))
    for i in range(5):
        op, loss = sess.run((train_op, total_loss))
        output,_ = sess.run((model_output, predictions))
        print(output["detection_boxes"])
        print("#"*20)
        print("#"*20)
#         loss_history.append(loss)
#         smoothed_loss = mean(loss_history[-50:])
#         progressbar.set_description("Loss: {0:.4f}".format(smoothed_loss))

[GPU 00] Restoring pretrained weights (265 Tensors) from: ../resnet/resnet-nhwc-2018-02-07/model.ckpt-112603
[[[2.9999995e-01 2.9999995e-01 2.9999995e-01 2.9999995e-01]
  [2.9999995e-01 2.9999995e-01 2.9999995e-01 2.9999995e-01]
  [2.9999995e-01 2.9999995e-01 2.9999995e-01 2.9999995e-01]
  ...
  [2.9999995e-01 2.9999995e-01 2.9999995e-01 2.9999995e-01]
  [2.9999995e-01 2.9999995e-01 2.9999995e-01 2.9999995e-01]
  [2.9999995e-01 2.9999995e-01 2.9999995e-01 2.9999995e-01]]

 [[7.3333335e-01 7.3333335e-01 7.3333335e-01 7.3333335e-01]
  [7.3333335e-01 7.3333335e-01 7.3333335e-01 7.3333335e-01]
  [7.3333335e-01 7.3333335e-01 7.3333335e-01 7.3333335e-01]
  ...
  [7.3333335e-01 7.3333335e-01 7.3333335e-01 7.3333335e-01]
  [7.3333335e-01 7.3333335e-01 7.3333335e-01 7.3333335e-01]
  [7.3333335e-01 7.3333335e-01 7.3333335e-01 7.3333335e-01]]

 [[9.1242796e-04 8.1263620e-01 8.1263620e-01 8.1263620e-01]
  [8.1263620e-01 8.1263620e-01 8.1263620e-01 8.1263620e-01]
  [8.1263620e-01 8.1263620e-01 8.12

[[[1.0992367  1.0992367  1.0992367  1.0992367 ]
  [1.0992367  1.0992367  1.0992367  1.0992367 ]
  [1.0992367  1.0992367  1.0992367  1.0992367 ]
  ...
  [1.0992367  1.0992367  1.0992367  1.0992367 ]
  [1.0992367  1.0992367  1.0992367  1.0992367 ]
  [1.0992367  1.0992367  1.0992367  1.0992367 ]]

 [[0.29999995 0.29999995 0.29999995 0.29999995]
  [0.29999995 0.29999995 0.29999995 0.29999995]
  [0.29999995 0.29999995 0.29999995 0.29999995]
  ...
  [0.29999995 0.29999995 0.29999995 0.29999995]
  [0.29999995 0.29999995 0.29999995 0.29999995]
  [0.29999995 0.29999995 0.29999995 0.29999995]]

 [[0.5435992  0.         0.5435992  0.5435992 ]
  [0.5435992  0.         0.5435992  0.5435992 ]
  [0.5435992  0.         0.5435992  0.5435992 ]
  ...
  [0.5435992  0.5435992  0.5435992  0.5435992 ]
  [0.5435992  0.5435992  0.5435992  0.5435992 ]
  [0.5435992  0.5435992  0.5435992  0.5435992 ]]

 [[0.0055455  0.         0.8325992  0.8325992 ]
  [0.8325992  0.         0.8325992  0.8325992 ]
  [0.8325992  0.